In [87]:
import numpy as np
import math as math
from numpy.typing import NDArray
import matplotlib.pyplot as plt
import random
from PIL import Image, ImageDraw, ImageFilter
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import skimage.measure



In [88]:
def dec2twocom(decimal_number):
    decimal_number = math.floor(decimal_number*(2**12))
    
    if decimal_number < -33554432 or decimal_number > 33554431:
        raise ValueError("Input out of 26-bit 2's complement range")

    if decimal_number < 0:
        decimal_number = (1 << 26) + decimal_number
    binary_str = format(decimal_number, '026b')

    return binary_str


In [89]:
dec2twocom(0.003)


'00000000000000000000001100'

In [90]:
n = 10000

images = []
labels = []
for i in range(n):
    width, height = 81, 81
    image = Image.new("1", (width, height), 0)
    draw = ImageDraw.Draw(image)
    x1, y1 = 0, random.randint(height//3, height//1.5)
    x2, y2 = width-1, random.randint(height//3, height//1.5)
    draw.line((x1, y1, x2, y2), fill=1 , width=1)
    image.save("random_line_image.png")
    convolved_image = Image.open(r"random_line_image.png").convert('RGB')
    rad = 10
    convolved_image = convolved_image.filter(ImageFilter.GaussianBlur(radius=rad))   
    convolved_image.save("convolved_image.png")
    convolved_array = 2.5*rad*np.array(convolved_image.convert('L'))
    small_array = skimage.measure.block_reduce(convolved_array, (27,27), np.average)
    # mean = 0
    # var = 256
    # sigma = var ** 0.5
    # gauss = np.random.normal(mean, sigma, small_array.shape)
    # gauss = gauss.reshape(small_array.shape)
    # noisy = small_array + gauss
    noisy = small_array
    if random.randint(0,1) == 0:
        images.append(np.array(noisy))
        labels.append(0)
    else:
        images.append(np.transpose(np.array(noisy)))
        labels.append(1)
images = np.array(images)
labels = np.array(labels)

In [91]:
image_input = []
for image in images:
    x = list(np.floor(image.reshape(-1)/32))
    x.append(1)
    image_input.append(np.array(x))
print(labels)

[1 1 0 ... 1 0 1]


In [92]:
split = 5000
images_train = image_input[:split]
images_test = image_input[split:]
labels_train = labels[:split]
labels_test = labels[split:]


In [148]:
class nueron_layer2:
    def __init__(self):
        self.act_out = 0
        self.lin_out = 0
        self.weights = [random.uniform(0.01, 0.1) for _ in range(9)]
    
    def activation(self): # Implement Hard Sigmoid
        return np.clip((self.lin_out + 1) / 2, 0, 1)
        # print(self.lin_out + 1)
        # return 0.8/(1 + math.exp(-self.lin_out))
    
    def forward_pass(self, input: np.array):#size 8 
        self.lin_out = np.dot(input, self.weights)
        self.act_out = self.activation()
        

In [94]:
class nueron_layer1:
    def __init__(self):
        self.act_out = 0
        self.lin_out = 0
        self.weights = [random.uniform(0.01, 0.1) for _ in range(10)]
    
    def activation(self): # Implement ReLu
        return np.clip(self.lin_out, 0, None)
    
    def forward_pass(self, input: np.array):#size 1
        self.lin_out = np.dot(input, self.weights)
        self.act_out = self.activation()

In [95]:
def der_hard_sigmoid(x):
    if x > 1 or x < -1:
        return 0
    elif x > -1 and x < 1:
        return 1/2

    #return x*(1-x)

def der_ReLu(x):

    if x > 0 :
        return 1
    elif x < 0:
        return 0


In [149]:
class nn:

    def __init__(self):
        self.n2a = nueron_layer2()
        self.n1 = [nueron_layer1() for i in range(8)]

    def initalise_trainingdata(self, input:NDArray, labels_nn, lr, niter):
        self.n = len(input)
        self.input = input
        self.labels = labels_nn
        self.learning_rate = lr
        self.niter = niter # number of iterations in one epoch

    
    def ForwardPassNueralNet(self, index=0):       
        
        
        for i in range(8):      
            self.n1[i].forward_pass(np.array(self.input[index]))
        n2a_input = [j.act_out for j in self.n1]        
        n2a_input.append(1)
        
        self.n2a.forward_pass(input=n2a_input)


    def gradient_calc(self): 
    
    # Square Loss ~ (output[i] - label[i])**2 ~ Is The Loss Function 

        
        gradient_vector = np.zeros(8*10+9) # first 9 are parameters related to the only nueron in nueron layer 2
                                    
        assert(len(self.labels) == len(self.input))
        for i in range(len(self.input)):
            self.ForwardPassNueralNet(i)
           
            for param in range(8):
                gradient_vector[param] += 2*(self.n2a.act_out - self.labels[i])*der_hard_sigmoid(self.n2a.lin_out)*self.n1[param].act_out   
            gradient_vector[8] += 2*(self.n2a.act_out - self.labels[i])*der_hard_sigmoid(self.n2a.lin_out)*1 # for the bias term
            
            for row in range(8):
                for col in range(10):
                    gradient_vector[9+row*10+col] += 2*(self.n2a.act_out - self.labels[i])*der_hard_sigmoid(self.n2a.lin_out)*self.n2a.weights[row]*der_ReLu(self.n1[row].lin_out)*self.input[i][col]
        return gradient_vector
        
    

    def gradient_descent(self):


        for _ in range(self.niter):
            gradients = self.gradient_calc()
            
            vec1 = self.n2a.weights
            vec2 = np.array([self.n1[i].weights for i in range(len(self.n1))])
            
            weight_vec = np.concatenate((vec1, vec2.reshape(-1)))           
            
            new_weight_vec = weight_vec - self.learning_rate*gradients
            assert(len(new_weight_vec) == 89)
            
            #assign the updates weights now:
            self.n2a.weights = np.array(new_weight_vec[0:9])
            for row in range(8):
                for col in range(10):
                    self.n1[row].weights[col] = weight_vec[9 + 10*row + col]
                    
            

    def calc_loss(self):
        output = []
        # first 8 are parameters related to the only nueron in nueron layer 2
        # last 8 are parameters related to the 8 nueron in nueron layer 1
        
        for i in range(len(self.input)):
            self.ForwardPassNueralNet(i)
            output.append(self.n2a.act_out)
        
            print((self.labels[i], output[i]))
        return sum([(self.labels[i] - output[i])**2 for i in range(len(output))])
    

    

        



In [152]:
my_nn = nn()

for _ in range(5):
    batch_size = 10
    n_ters = 500
    for row in range(n_ters):
        my_nn.initalise_trainingdata(images_train[row*10:(row+1)*10], labels_train[row*10:(row+1)*10],lr= 0.001, niter=1)
        
        my_nn.gradient_descent()



In [164]:

my_nn.initalise_trainingdata(images_test[10:20], labels_test[10:20], 0.0001, 1)
my_nn.n2a.weights = [-0.85484332, -0.25137118, -0.53657609,  0.69090954,  0.41767282, -0.01276339, 0.51725959,  0.24410813,  0.02319168]
loss = my_nn.calc_loss()
print(loss)
print(sum)


(1, 0.7907093560914835)
(1, 0.7975822730471304)
(0, 0.30969784551010443)
(0, 0.2591802106983149)
(1, 0.787769266575129)
(0, 0.2382925988054151)
(1, 0.8276778300446312)
(0, 0.39498292412870106)
(1, 0.7875717268550912)
(1, 0.7311033457365321)
0.6528255163159298
<built-in function sum>


In [172]:
binary_weights_layer1 = []
for i in range(8):
    converted = [dec2twocom(x) for x in my_nn.n1[i].weights]
    binary_weights_layer1.append(converted)
    print(converted)

    

['00000000000000000100001011', '00000000000000000001100000', '00000000000000000011010001', '00000000000000000101000010', '00000000000000000001111101', '00000000000000000011101100', '00000000000000000001001000', '00000000000000000000110010', '00000000000000000010000111', '00000000000000000010111000']
['00000000000000000000101001', '00000000000000000011000111', '00000000000000000011100100', '00000000000000000011010110', '00000000000000000100101010', '00000000000000000010000000', '00000000000000000010111010', '00000000000000000001010010', '00000000000000000001101101', '00000000000000000010010100']
['00000000000000000110011000', '00000000000000000011000100', '00000000000000000011110100', '00000000000000000101100111', '00000000000000000100100001', '00000000000000000010010001', '00000000000000000101010001', '00000000000000000011001001', '00000000000000000011100010', '00000000000000000010111001']
['00000000000000000110000011', '00000000000000000010001101', '00000000000000000001111111', '00000

In [186]:
print(my_nn.input[-4])
print(my_nn.labels[-4])
print(my_nn.ForwardPassNueralNet(-1))
print(my_nn.n2a.act_out)

[2. 4. 0. 0. 5. 0. 0. 5. 1. 1.]
1
None
0.7311033457365321


In [173]:
binary_weights_layer2 = [dec2twocom(x) for x in my_nn.n2a.weights] 
print(binary_weights_layer2)
print(my_nn.n2a.weights)

    

['11111111111111001001010010', '11111111111111101111111010', '11111111111111011101101010', '00000000000000101100001101', '00000000000000011010101110', '11111111111111111111001011', '00000000000000100001000110', '00000000000000001111100111', '00000000000000000001011110']
[-0.85484332, -0.25137118, -0.53657609, 0.69090954, 0.41767282, -0.01276339, 0.51725959, 0.24410813, 0.02319168]


In [171]:
for j in range(8):
    print(my_nn.n1[i].act_out)


1.2067153580159475
1.2067153580159475
1.2067153580159475
1.2067153580159475
1.2067153580159475
1.2067153580159475
1.2067153580159475
1.2067153580159475


In [102]:
my_nn.initalise_trainingdata(input_nn, labels, lr, niter)
my_nn.gradient_calc()
my_nn.gradient_descent()
print(my_nn.calc_loss())

NameError: name 'input_nn' is not defined

In [ ]:
np.dot([0.69381075, 0.77043915 ,0.15036522 ,0.33325319, 0.39468583, 0.34754608,
 0.71602594, 0.99910301, 0.13833847, 0.41147616], [2 ,2 ,4, 5, 5, 4, 1, 0, 0, 1])

9.6873422